In [51]:
from model import river_model, losses, dot_prod_attention
from data import data_generation, batch_creator, gp_kernels
from keras.callbacks import ModelCheckpoint
from helpers import helpers, masks, metrics
import matplotlib.pyplot as plt
import tensorflow_addons as tfa
from inference import infer
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib 
import time
import keras
plt.style.use('ggplot')

In [52]:
attributes_numeric = pd.read_csv('/Users/omernivron/Downloads/Data_riverflow/att_numeric')

In [3]:
attributes_numeric.iloc[:, 1:] = (attributes_numeric.iloc[:, 1:] - np.mean(attributes_numeric.iloc[:, 1:], axis = 0)) / np.std(attributes_numeric.iloc[:, 1:], axis = 0)

In [4]:
arr_pp = np.load('/Users/omernivron/Downloads/river_processed.npy')

In [5]:
arr_pe = np.load('/Users/omernivron/Downloads/river_processed_te.npy')

In [43]:
save_dir = '/Users/omernivron/Downloads/GPT_river_mha'

In [44]:
loss_object = tf.keras.losses.MeanSquaredError()
train_loss = tf.keras.metrics.Mean(name='train_loss')
test_loss = tf.keras.metrics.Mean(name='test_loss')
m_tr = tf.keras.metrics.Mean()
m_te = tf.keras.metrics.Mean()

In [45]:
@tf.function
def train_step(decoder, optimizer_c, train_loss, m_tr, token_pos, time_pos, time_pos2, pos, tar, pos_mask):
    '''
    A typical train step function for TF2. Elements which we wish to track their gradient
    has to be inside the GradientTape() clause. see (1) https://www.tensorflow.org/guide/migrate 
    (2) https://www.tensorflow.org/tutorials/quickstart/advanced
    ------------------
    Parameters:
    pos (np array): array of positions (x values) - the 1st/2nd output from data_generator_for_gp_mimick_gpt
    tar (np array): array of targets. Notice that if dealing with sequnces, we typically want to have the targets go from 0 to n-1. The 3rd/4th output from data_generator_for_gp_mimick_gpt  
    pos_mask (np array): see description in position_mask function
    ------------------    
    '''
    tar_inp = tar[:, :-1]
    tar_real = tar[:, 1:]
    combined_mask_tar = masks.create_masks(tar_inp)
    with tf.GradientTape(persistent=True) as tape:
        pred, pred_log_sig = decoder(token_pos, time_pos, time_pos2, pos, tar_inp, True, pos_mask, combined_mask_tar)
#         print('pred: ')
#         tf.print(pred_sig)

        loss, mse, mask = losses.loss_function(tar_real, pred, pred_log_sig)


    gradients = tape.gradient(loss, decoder.trainable_variables)
#     tf.print(gradients)
# Ask the optimizer to apply the processed gradients.
    optimizer_c.apply_gradients(zip(gradients, decoder.trainable_variables))
    train_loss(loss)
    m_tr.update_state(mse, mask)
#     b = decoder.trainable_weights[0]
#     tf.print(tf.reduce_mean(b))
    return tar_inp, tar_real, pred, pred_log_sig, mask

In [46]:
@tf.function
def test_step(decoder, test_loss, m_te, token_pos_te, time_pos_te, time_pos2_te, pos_te, tar_te, pos_mask_te):
    '''
    
    ---------------
    Parameters:
    pos (np array): array of positions (x values) - the 1st/2nd output from data_generator_for_gp_mimick_gpt
    tar (np array): array of targets. Notice that if dealing with sequnces, we typically want to have the targets go from 0 to n-1. The 3rd/4th output from data_generator_for_gp_mimick_gpt  
    pos_mask_te (np array): see description in position_mask function
    ---------------
    
    '''
    tar_inp_te = tar_te[:, :-1]
    tar_real_te = tar_te[:, 1:]
    combined_mask_tar_te = masks.create_masks(tar_inp_te)
  # training=False is only needed if there are layers with different
  # behavior during training versus inference (e.g. Dropout).
    pred_te, pred_log_sig_te = decoder(token_pos_te, time_pos_te, time_pos2_te, pos_te, tar_inp_te, False, pos_mask_te, combined_mask_tar_te)
    t_loss, t_mse, t_mask = losses.loss_function(tar_real_te, pred_te, pred_log_sig_te)
    test_loss(t_loss)
    m_te.update_state(t_mse, t_mask)
    return tar_real_te, pred_te, pred_log_sig_te, t_mask

In [47]:
tf.keras.backend.set_floatx('float64')

In [48]:
t1_tr = arr_pp[1::5]; t2_tr = arr_pp[2::5]
tar_tr = arr_pp[0::5];
token_tr = arr_pp[3::5]; basin_l_tr = arr_pp[4::5] 

In [49]:
t1_te = arr_pe[1::5]; t2_te = arr_pe[2::5]
tar_te = arr_pe[0::5];
token_te = arr_pe[3::5]; basin_l_te = arr_pe[4::5] 

In [50]:
if __name__ == '__main__':
    writer = tf.summary.create_file_writer(save_dir + '/logs/')
    optimizer_c = tf.keras.optimizers.Adam(0.0004)
    decoder = river_model.Decoder(16)
    EPOCHS = 500
    batch_s  = 64
    run = 0; step = 0
    num_batches = int(tar_tr.shape[0] / batch_s)
    tf.random.set_seed(1)
    ckpt = tf.train.Checkpoint(step=tf.Variable(1), optimizer = optimizer_c, net = decoder)
    main_folder = "/Users/omernivron/Downloads/GPT_river_mha/ckpt/check_"
    folder = main_folder + str(run); helpers.mkdir(folder)
    #https://www.tensorflow.org/guide/checkpoint
    manager = tf.train.CheckpointManager(ckpt, folder, max_to_keep=3)
    ckpt.restore(manager.latest_checkpoint)
    if manager.latest_checkpoint:
        print("Restored from {}".format(manager.latest_checkpoint))
    else:
        print("Initializing from scratch.")

    with writer.as_default():
        for epoch in range(EPOCHS):
            start = time.time()

            for batch_n in range(num_batches):
                m_tr.reset_states(); train_loss.reset_states()
                m_te.reset_states(); test_loss.reset_states()
                batch_tok_pos_tr, batch_tim_pos_tr, batch_tim_pos_tr2, batch_pos_tr, batch_tar_tr, _ = batch_creator.create_batch_river(token_tr, t1_tr, t2_tr, basin_l_tr,  attributes_numeric, tar_tr, batch_s=64)
                batch_pos_mask = masks.position_mask(batch_tok_pos_tr)
                tar_inp, tar_real, pred, pred_log_sig, mask = train_step(decoder, optimizer_c, train_loss, m_tr, batch_tok_pos_tr, batch_tim_pos_tr, batch_tim_pos_tr2, batch_pos_tr, batch_tar_tr, batch_pos_mask)

                if batch_n % 100 == 0:
                    batch_tok_pos_te, batch_tim_pos_te, batch_tim_pos_te2, batch_pos_te, batch_tar_te, _ = batch_creator.create_batch_river(token_te, t1_te, t2_te, basin_l_te,  attributes_numeric, tar_te, batch_s=64)
                    batch_pos_mask_te = masks.position_mask(batch_tok_pos_te)
                    tar_real_te, pred_te, pred_log_sig_te, t_mask = test_step(decoder, test_loss, m_te, batch_tok_pos_te, batch_tim_pos_te, batch_tim_pos_te2, batch_pos_te, batch_tar_te, batch_pos_mask_te)
                    helpers.print_progress(epoch, batch_n, train_loss.result(), test_loss.result(), m_te.result())
                    helpers.tf_summaries(run, step, train_loss.result(), test_loss.result(), m_tr.result(), m_te.result())
                    manager.save()
                step += 1
                ckpt.step.assign_add(1)

            print ('Time taken for 1 epoch: {} secs\n'.format(time.time() - start))

Already exists
Restored from /Users/omernivron/Downloads/GPT_river_mha/ckpt/check_0/ckpt-219
Epoch 0 batch 0 train Loss 1.7003 test Loss 1.5920 with MSE metric 2.9140
Epoch 0 batch 100 train Loss 1.9576 test Loss 1.5821 with MSE metric 3.0140
Time taken for 1 epoch: 68.68693995475769 secs

Epoch 1 batch 0 train Loss 1.7496 test Loss 2.0357 with MSE metric 10.5334
Epoch 1 batch 100 train Loss 1.7829 test Loss 1.7077 with MSE metric 5.3546
Time taken for 1 epoch: 65.68280506134033 secs

Epoch 2 batch 0 train Loss 2.0039 test Loss 1.6149 with MSE metric 3.1979
Epoch 2 batch 100 train Loss 1.9334 test Loss 1.8944 with MSE metric 8.1001
Time taken for 1 epoch: 66.18736481666565 secs

Epoch 3 batch 0 train Loss 1.8210 test Loss 1.6747 with MSE metric 3.7088
Epoch 3 batch 100 train Loss 1.7148 test Loss 1.5868 with MSE metric 3.3897
Time taken for 1 epoch: 66.06295394897461 secs

Epoch 4 batch 0 train Loss 1.9319 test Loss 1.5569 with MSE metric 1.9793
Epoch 4 batch 100 train Loss 1.9670 test

Time taken for 1 epoch: 67.43184781074524 secs

Epoch 41 batch 0 train Loss 1.7189 test Loss 2.1640 with MSE metric 12.2645
Epoch 41 batch 100 train Loss 1.8803 test Loss 1.6014 with MSE metric 3.5047
Time taken for 1 epoch: 64.68967700004578 secs

Epoch 42 batch 0 train Loss 2.2002 test Loss 1.6405 with MSE metric 3.8881
Epoch 42 batch 100 train Loss 1.9717 test Loss 1.4749 with MSE metric 1.7314
Time taken for 1 epoch: 65.55017495155334 secs

Epoch 43 batch 0 train Loss 1.7075 test Loss 1.6975 with MSE metric 4.8254
Epoch 43 batch 100 train Loss 1.8016 test Loss 1.7483 with MSE metric 5.6994
Time taken for 1 epoch: 66.66386198997498 secs

Epoch 44 batch 0 train Loss 2.0605 test Loss 1.5582 with MSE metric 2.5423
Epoch 44 batch 100 train Loss 1.6530 test Loss 1.7476 with MSE metric 5.9125
Time taken for 1 epoch: 66.63962292671204 secs

Epoch 45 batch 0 train Loss 1.8703 test Loss 1.5001 with MSE metric 1.7336
Epoch 45 batch 100 train Loss 1.9103 test Loss 1.5535 with MSE metric 2.5684

Time taken for 1 epoch: 64.99147915840149 secs

Epoch 82 batch 0 train Loss 1.8911 test Loss 1.6046 with MSE metric 3.6536
Epoch 82 batch 100 train Loss 1.6564 test Loss 2.2140 with MSE metric 13.9296
Time taken for 1 epoch: 65.14191389083862 secs

Epoch 83 batch 0 train Loss 2.3785 test Loss 2.0898 with MSE metric 10.7116
Epoch 83 batch 100 train Loss 1.7044 test Loss 1.5297 with MSE metric 1.8545
Time taken for 1 epoch: 65.07447814941406 secs

Epoch 84 batch 0 train Loss 1.8248 test Loss 1.5480 with MSE metric 3.3737
Epoch 84 batch 100 train Loss 1.6089 test Loss 1.4718 with MSE metric 2.0628
Time taken for 1 epoch: 65.38776898384094 secs

Epoch 85 batch 0 train Loss 1.6787 test Loss 1.5656 with MSE metric 3.0915
Epoch 85 batch 100 train Loss 2.0155 test Loss 1.4797 with MSE metric 2.3750
Time taken for 1 epoch: 66.9913101196289 secs

Epoch 86 batch 0 train Loss 1.8378 test Loss 1.6528 with MSE metric 4.3781
Epoch 86 batch 100 train Loss 1.6997 test Loss 1.4512 with MSE metric 1.5587

Time taken for 1 epoch: 65.50670123100281 secs

Epoch 123 batch 0 train Loss 2.5393 test Loss 1.6965 with MSE metric 5.0370
Epoch 123 batch 100 train Loss 1.6616 test Loss 1.4926 with MSE metric 1.8381
Time taken for 1 epoch: 65.44403100013733 secs

Epoch 124 batch 0 train Loss 1.8633 test Loss 1.5024 with MSE metric 2.1573
Epoch 124 batch 100 train Loss 1.8734 test Loss 1.5034 with MSE metric 1.5040
Time taken for 1 epoch: 65.45074510574341 secs

Epoch 125 batch 0 train Loss 1.6104 test Loss 1.9068 with MSE metric 8.3260
Epoch 125 batch 100 train Loss 1.6162 test Loss 1.6224 with MSE metric 4.3298
Time taken for 1 epoch: 65.29555988311768 secs

Epoch 126 batch 0 train Loss 1.8754 test Loss 1.8883 with MSE metric 7.9510
Epoch 126 batch 100 train Loss 1.6091 test Loss 1.4982 with MSE metric 2.2169
Time taken for 1 epoch: 65.35960078239441 secs

Epoch 127 batch 0 train Loss 1.7532 test Loss 1.6907 with MSE metric 4.8703
Epoch 127 batch 100 train Loss 1.5517 test Loss 1.5480 with MSE metr

Epoch 163 batch 100 train Loss 2.0245 test Loss 1.4164 with MSE metric 1.6491
Time taken for 1 epoch: 67.72444891929626 secs

Epoch 164 batch 0 train Loss 1.7513 test Loss 1.5969 with MSE metric 3.6399
Epoch 164 batch 100 train Loss 1.6079 test Loss 1.5140 with MSE metric 2.4270
Time taken for 1 epoch: 66.69918322563171 secs

Epoch 165 batch 0 train Loss 1.6897 test Loss 1.4730 with MSE metric 1.5551
Epoch 165 batch 100 train Loss 1.7291 test Loss 1.5715 with MSE metric 3.4866
Time taken for 1 epoch: 65.77011299133301 secs

Epoch 166 batch 0 train Loss 1.7233 test Loss 1.5141 with MSE metric 2.3748
Epoch 166 batch 100 train Loss 1.5567 test Loss 1.4909 with MSE metric 2.6818
Time taken for 1 epoch: 67.3675811290741 secs

Epoch 167 batch 0 train Loss 1.8544 test Loss 1.6479 with MSE metric 4.0020
Epoch 167 batch 100 train Loss 1.7361 test Loss 1.6131 with MSE metric 3.5311
Time taken for 1 epoch: 65.65760588645935 secs

Epoch 168 batch 0 train Loss 1.7952 test Loss 1.6032 with MSE metri

KeyboardInterrupt: 

In [21]:
df = pd.read_csv('/Users/omernivron/Downloads/Data_riverflow/river_flow_processed.csv')

In [25]:
basin = pd.read_csv('/Users/omernivron/Downloads/Data_riverflow/basin_list.txt', header= None)

In [26]:
basin

,0
0,1022500
1,1031500
2,1047000
3,1052500
4,1054200
...,...
526,11482500
527,11522500
528,11523200
529,11528700


In [32]:
sum((df[df['basin'] == 1022500]['OBS_RUN'] - np.mean(df[df['basin'] == 1022500]['OBS_RUN']))**2) / len(df[df['basin'] == 1022500]['OBS_RUN'])

6.181555968690538

In [ ]:
1- 4.4 / (sum((df[df['basin'] == 1022500]['OBS_RUN'] - np.mean(df[df['basin'] == 1022500]['OBS_RUN']))**2) / len(df[df['basin'] == 1022500]['OBS_RUN']))